## Goals:

**1. Determine drivers of sentiment in the flavor text in Magic: The Gathering cards.**

**2. Develop a model to predict the sentiment of flavor text in Magic: The Gathering cards.**

In [1]:
# imports

import pandas as pd
import numpy as np

import prepare as p

In [2]:
df = pd.read_csv('cards.csv')
df.columns

/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5,8,14,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['index', 'id', 'artist', 'borderColor', 'colorIdentity',
       'colorIndicator', 'colors', 'convertedManaCost', 'duelDeck',
       'edhrecRank', 'faceConvertedManaCost', 'flavorText', 'frameEffect',
       'frameEffects', 'frameVersion', 'hand', 'hasFoil', 'hasNoDeckLimit',
       'hasNonFoil', 'isAlternative', 'isArena', 'isBuyABox', 'isDateStamped',
       'isFullArt', 'isMtgo', 'isOnlineOnly', 'isOversized', 'isPaper',
       'isPromo', 'isReprint', 'isReserved', 'isStarter', 'isStorySpotlight',
       'isTextless', 'isTimeshifted', 'layout', 'leadershipSkills', 'life',
       'loyalty', 'manaCost', 'mcmId', 'mcmMetaId', 'mtgArenaId', 'mtgoFoilId',
       'mtgoId', 'multiverseId', 'name', 'names', 'number', 'originalText',
       'originalType', 'otherFaceIds', 'power', 'printings', 'purchaseUrls',
       'rarity', 'scryfallId', 'scryfallIllustrationId', 'scryfallOracleId',
       'setCode', 'side', 'subtypes', 'supertypes', 'tcgplayerProductId',
       'text', 'toughness', 

In [3]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)

In [4]:
df = p.prepare_mgt(p.wrangle_mtg())

df.head(20)

/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (5,8,14,38) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,colorIdentity,types,convertedManaCost,rarity,flavorText
1,Blue,Creature,3.0,uncommon,They brandish their latest theories as warriors would wield weapons.
3,Black,Instant,3.0,common,"One rarely notices a heartbeat, save when it is stolen."
4,Green,Instant,2.0,common,"The power of the wild, concentrated in a single charge."
5,Black,Sorcery,4.0,uncommon,"In the aftermath of war, when the slaying is long done, the greatest miseries come home to roost."
6,Blue,Creature,5.0,uncommon,"""The East Wind, an interloper in the dominions of Westerly Weather, is an impassive-faced tyrant with a sharp poniard held behind his back for a treacherous stab."" —Joseph Conrad, The Mirror of the Sea"
7,Blue,Creature,5.0,uncommon,"""The East Wind, an interloper in the dominions of Westerly Weather, is an impassive-faced tyrant with a sharp poniard held behind his back for a treacherous stab.""\n—Joseph Conrad, The Mirror of the Sea"
8,Blue,Creature,3.0,rare,"""Life is a game. The only thing that matters is whether you're a pawn or a player."""
9,Red,Creature,4.0,common,"""They who challenge a minotaur enjoy the taste of their own blood."" —Mirri of the Weatherlight"
10,Red,Creature,4.0,common,"""They who challenge a minotaur enjoy the taste of their own blood."" —Mirri of the Weatherlight"
12,White,Creature,7.0,uncommon,"""The will of all, by my hand done."""


In [5]:
df = df.groupby('flavorText').agg('max').reset_index()
df = df[['colorIdentity','types','convertedManaCost','rarity','flavorText']]

In [6]:
df.dtypes

colorIdentity        object 
types                object 
convertedManaCost    float64
rarity               object 
flavorText           object 
dtype: object

In [7]:
df = df.astype({"flavorText":'str'})

In [8]:
df.dtypes

colorIdentity        object 
types                object 
convertedManaCost    float64
rarity               object 
flavorText           object 
dtype: object

In [9]:
import re

def replace(value):
    
    value = re.sub(r"[/]",'', value)
    
    return value


    
df['flavorText'] = df.flavorText.apply(replace)



   

In [10]:
value = "\". . . When the trees bow down their heads, The wind is passing by.\" —Christina Rossetti, \"Who Has Seen the Wind?"


def erase_end(value):
    
    count = 0
    index = 0
    capture = False

    for letter in value:
    
        if count != 0:
            if letter == '\"':
                capture = True
                index = count
                break
            
        count += 1
    
    if capture == True: 
        
        return value[:index] + '"'
    
    else:
        return value

df['flavorText'] = df.flavorText.apply(erase_end)

df.head(10)

,colorIdentity,types,convertedManaCost,rarity,flavorText
0,Green,Creature,5.0,common,""" . . . And the third little boar built his house out of rootwalla plates . . . ."""
1,Black,Creature,1.0,common,""" . . . Cao Pi, Cao Rui, Fang, Mao, and briefly, Huan— The Sima took the empire in their turn. . . ."""
2,Blue,Creature,5.0,uncommon,""" . . . When the trees bow down their heads, The wind is passing by."""
3,Blue,Creature,5.0,uncommon,""" . . . When the trees bow down their heads, The wind is passing by."""
4,White,Creature,4.0,uncommon,""" . . . and you must also apply for an application license, file documents 136(iv) and 22-C and -D in triplicate, pay all requisite fees, request a . . ."""
5,Green,Creature,4.0,common,"""'Air superiority?' Not while our archers scan the skies."""
6,Black,Enchantment,2.0,rare,"""'Davvol, blast those elves.' 'Davvol, transport those troops.' No one cares that today is my birthday."""
7,White,Creature,3.0,common,"""'Do you know, I always thought Unicorns were fabulous monsters, too? I never saw one alive before!' 'Well, now that we have seen each other,' said the Unicorn, 'if you'll believe in me, I'll believe in you.'"""
8,White,Creature,3.0,common,"""'Do you know, I always thought Unicorns were fabulous monsters, too? I never saw one alive before!' 'Well, now that we have seen each other,' said the Unicorn, 'if you'll believe in me, I'll believe in you.'"""
9,Blue,Creature,2.0,common,"""'Fraid of heights? Ride a horse. 'Fraid of missing out? Rein up a drake."""


In [11]:
df.shape

(12716, 5)

In [12]:
df = df.groupby('flavorText').agg('max').reset_index()
df = df[['colorIdentity','types','convertedManaCost','rarity','flavorText']]

In [13]:
df.shape

(12458, 5)

In [14]:
def remove_space(value):
    
    return value.strip()

df['flavorText'] = df.flavorText.apply(erase_end)

df.head(10)

,colorIdentity,types,convertedManaCost,rarity,flavorText
0,Green,Creature,5.0,common,""" . . . And the third little boar built his house out of rootwalla plates . . . ."""
1,Black,Creature,1.0,common,""" . . . Cao Pi, Cao Rui, Fang, Mao, and briefly, Huan— The Sima took the empire in their turn. . . ."""
2,Blue,Creature,5.0,uncommon,""" . . . When the trees bow down their heads, The wind is passing by."""
3,Blue,Creature,5.0,uncommon,""" . . . When the trees bow down their heads, The wind is passing by."""
4,White,Creature,4.0,uncommon,""" . . . and you must also apply for an application license, file documents 136(iv) and 22-C and -D in triplicate, pay all requisite fees, request a . . ."""
5,Green,Creature,4.0,common,"""'Air superiority?' Not while our archers scan the skies."""
6,Black,Enchantment,2.0,rare,"""'Davvol, blast those elves.' 'Davvol, transport those troops.' No one cares that today is my birthday."""
7,White,Creature,3.0,common,"""'Do you know, I always thought Unicorns were fabulous monsters, too? I never saw one alive before!' 'Well, now that we have seen each other,' said the Unicorn, 'if you'll believe in me, I'll believe in you.'"""
8,Blue,Creature,2.0,common,"""'Fraid of heights? Ride a horse. 'Fraid of missing out? Rein up a drake."""
9,Black,Instant,1.0,common,"""'Here lies our beloved Gartu, the gentlest of all men.' Ooh, I love irony."""


In [15]:
df.shape

(12458, 5)

In [19]:
df = df.groupby('flavorText').agg('max').reset_index()
df = df[['colorIdentity','types','convertedManaCost','rarity','flavorText']]

In [17]:
df.shape

(12458, 5)

In [20]:
df.flavorText.value_counts()

Kill a gnarlid with your first blow, or it'll cheerfully show you how it's done.                                                                                                                                                                                                                                  1
She walks in the twilight, her steps make no sound, Her feet leave no tracks on the dew-covered ground. Her hand gently beckons, she whispers your name— But those who go with her are never the same.                                                                                                            1
"Picking locks is for beginners."                                                                                                                                                                                                                                                                                 1
"And when this land is wasted, where will we go?"                           

# Prepare

A CSV, containing an up to date breakdown of each Magic card that has been printed so far, was obtained from MTGJSON.com. The data was then read into a pandas dataframe.

In [18]:
df = pandas.read_csv('cards.csv')

NameError: name 'pandas' is not defined

The dataframe contained 50,412 rows and 71 columns

The process of preparing the data begins by selecting columns that are patentualy relivent to sentiment.

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df = df[['colorIdentity','types','convertedManaCost','rarity','flavorText','isPaper']]

In [ ]:
df = df[df.isPaper==1]
df = df.drop(columns='isPaper')
df.head()

In [ ]:
df = df[df.flavorText.notna()]

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
colors = ['W','U','B','R','G']

df = df.loc[df.colorIdentity.isin(colors)]

In [ ]:
df.shape

In [ ]:
df.types.value_counts()

In [ ]:
def convert_type(df,copy, change):
    
    for item in change:
        
        df['types'] = np.where(df['types'] == f'{item}', f'{copy}', df['types'])
        
        return df

In [ ]:
change = ['Artifact,Creature','Summon']
copy = 'Creature'

convert_type(df, copy, change)

df.types.value_counts()

In [ ]:
change = ['Land,Creature','Artifact,Land']
copy = 'Land'

convert_type(copy, change)

df.types.value_counts()

In [ ]:
change = ['Tribal,Instant']
copy = 'Instant'

convert_type(copy, change)

df.types.value_counts()

In [ ]:
change = ['Tribal,Sorcery']
copy = 'Sorcery'

convert_type(copy, change)

df.types.value_counts()

In [ ]:
change = ['Tribal,Enchantment']
copy = 'Enchantment'

convert_type(copy, change)

df.types.value_counts()

In [ ]:
change = ['instant']
copy = 'Instant'

convert_type(copy, change)

df.types.value_counts()

In [ ]:
types = ['Creature','Instant','Sorcery','Enchantment','Land','Artifact','Planeswalker']

df = df.loc[df.types.isin(types)]

df.head()